**Preliminary exploratory data analysis**

1. Demonstrate that the dataset can be read from the web into R 
2. Clean and wrangle your data into a tidy format

In [22]:
library(tidyverse)

In [66]:
url <- "https://raw.githubusercontent.com/churancc/Dementia_Project/main/oasis_longitudinal.csv"

dementia_df <- read_csv(url) |>
mutate(across(c("Group", "M/F", "Hand"), as_factor))

unique_names <- make.names(names(dementia_df), unique=TRUE)
colnames(dementia_df) <- unique_names
colnames(dementia_df)[c(3,6)] <- c("Diagnosis", "Sex")

head(dementia_df)

Rows: 373 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): Subject ID, MRI ID, Group, M/F, Hand
dbl (10): Visit, MR Delay, Age, EDUC, SES, MMSE, CDR, eTIV, nWBV, ASF

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Subject.ID,MRI.ID,Diagnosis,Visit,MR.Delay,Sex,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
<chr>,<chr>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2,27,0.0,1987,0.696,0.883
OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2,30,0.0,2004,0.681,0.876
OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NA,23,0.5,1678,0.736,1.046
OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NA,28,0.5,1738,0.713,1.010
OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NA,22,0.5,1698,0.701,1.034
OAS2_0004,OAS2_0004_MR1,Nondemented,1,0,F,R,88,18,3,28,0.0,1215,0.710,1.444


3. Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

In [67]:
class_count <- dementia_df |>
                    group_by(Subject.ID, Diagnosis) |>
                    summarize(Count = n())

class_count

`summarise()` has grouped output by 'Subject.ID'. You can override using the
`.groups` argument.


Subject.ID,Diagnosis,Count
<chr>,<fct>,<int>
OAS2_0001,Nondemented,2
OAS2_0002,Demented,3
OAS2_0004,Nondemented,2
OAS2_0005,Nondemented,3
OAS2_0007,Demented,3
OAS2_0008,Nondemented,2
OAS2_0009,Demented,2
OAS2_0010,Demented,2
OAS2_0012,Nondemented,3


4. Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.